In [1]:
import h2o

In [2]:
import os
os.getcwd()

'/home/jovyan/work'

In [3]:
h2o.init(ip='h2o-service')

Checking whether there is an H2O instance running at http://h2o-service:54321 . connected.


H2O_cluster_uptime:,29 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.102 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
from h2o.automl import H2OAutoML
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")
# Identify the response and set of predictors
y = "response"
x = list(train.columns)  #if x is defined as all columns except the response, then x is not required
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 20*60)
aml.train(x = x, y = y, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [6]:
# Print Leaderboard (ranked by xval metrics)
aml.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20210316_084408,0.788545,0.553722,0.805423,0.334178,0.43319,0.187653
StackedEnsemble_BestOfFamily_AutoML_20210316_084408,0.787227,0.554809,0.802605,0.329653,0.433609,0.188017
GBM_grid__1_AutoML_20210316_084408_model_12,0.782404,0.55895,0.798865,0.321832,0.435547,0.189701
GBM_grid__1_AutoML_20210316_084408_model_4,0.781921,0.559547,0.799567,0.339646,0.435868,0.189981
GBM_5_AutoML_20210316_084408,0.781486,0.558647,0.800219,0.314708,0.435708,0.189841
GBM_grid__1_AutoML_20210316_084408_model_10,0.781362,0.558282,0.801433,0.31042,0.435749,0.189877
GBM_grid__1_AutoML_20210316_084408_model_8,0.780498,0.559834,0.798931,0.329721,0.436222,0.190289
GBM_2_AutoML_20210316_084408,0.778929,0.56092,0.798517,0.331067,0.436975,0.190947
XGBoost_grid__1_AutoML_20210316_084408_model_9,0.778379,0.561948,0.796449,0.318358,0.437365,0.191288
GBM_1_AutoML_20210316_084408,0.778076,0.562233,0.796263,0.321418,0.43738,0.191302


In [7]:
# (Optional) Evaluate performance on a test set
perf = aml.leader.model_performance(test)
perf.auc()

0.7929771428341599